In [1]:
import pySldWrap.sw_tools as sw_tools
import importlib
from pathlib import Path

In [21]:
#reloads sw_tools module so changes in .py file actually work without restarting the kernel when running the cell blocks below
#reload causes all variables to be lost; do not do it in other cells besides this block
importlib.reload(sw_tools)

sw_tools.connect_sw("2024")  # open connection and pass Solidworks version

path = './Test_files/LS3.SLDPRT'
# path = Path(path)  # a path object can also be used for a number of functions
model = sw_tools.open_part(path)  # open the model, link is returned

In [22]:
sw_tools.close(path) 